# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part V: Evaluating the Model on the Test Set

In this final part of the tutorial, we will reload the model developed in Part IV and test it on the test `CandidateSet`.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass, and load the test set:

In [ ]:
from snorkel.models import candidate_subclass

Spouse          = candidate_subclass('Spouse', ['person1', 'person2'])
test_candidates = session.query(Spouse).filter(Spouse.split == 2).all()

## Automatically Creating Features
We also generate features for the test set.

In [ ]:
from snorkel.annotations import FeatureManager
feature_manager = FeatureManager(Spouse)

Note that we use the training features feature set, because those are the only features for which we have learned parameters. Features that were not encountered during training, e.g., a token that does not appear in the training set, are ignored, because we do not have any information about them.

To do so with the `FeatureManager`, we call update with the new `CandidateSet`, the name of the training `AnnotationKeySet`, and the value `False` for the parameter `extend_key_set` to indicate that the `AnnotationKeySet` should not be expanded with new `Feature` keys encountered during processing.

In [ ]:
%time F_test = feature_manager.update(session, False, split=2)

**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_test = feature_manager.load(session, split=2)

## Reloading the Discriminative Model

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
%time disc_model.load(session)

## Evaluating on the Test `CandidateSet`

First, we load the test set labels and gold candidates we made in Part III.

In [ ]:
from snorkel.annotations import AnnotatorLabelManager

anno_label_manager = AnnotatorLabelManager(Spouse)
L_gold_test        = anno_label_manager.load(session, annotator_name='gold', split=2)

In [ ]:
tp, fp, tn, fn = disc_model.score(session, F_test, L_gold_test)

## Viewing Examples
We can also view the results on the test `CandidateSet`,  You can view the true positives, false positives, true negatives, and false negatives using the `Viewer`.

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(tp, session, annotator_name="Tutorial Part V User")
else:
    sv = None

In [ ]:
sv

You've completed the introduction to Snorkel!